In [72]:
import pickle
test1 = pickle.load(open('./pickled_objects/test1.p','rb'))
probssentences = pickle.load(open('./pickled_objects/probssentences.p', 'rb'))
probssentences_train = pickle.load(open('./pickled_objects/probssentences_train.p', 'rb'))
train1 = pickle.load(open('./pickled_objects/train1.p','rb'))

In [21]:
#last sentence only
import numpy as np
preds = []
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0][-1]
    if probssentences[0][inx] > probssentences[1][inx]:
        preds.append(0)
    else:
        preds.append(1)
    

In [26]:
#standard averaging
import numpy as np
preds = []
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0]
    if np.mean(probssentences[0][inx]) > np.mean(probssentences[1][inx]):
        preds.append(0)
    else:
        preds.append(1)


In [79]:
#max
import numpy as np
preds = []
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0]
    if np.max(probssentences[0][inx]) > np.max(probssentences[1][inx]):
        preds.append(0)
    else:
        preds.append(1)


In [77]:
#apply sigmoidal function
import numpy as np
preds = []
k=10
for doc in [0]:#np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0]
    scores0 = probssentences[0][inx].astype(float)-0.5
    scores0 = 1./(1+np.exp(-k*scores0))
    scores1 = probssentences[1][inx].astype(float)-0.5
    scores1 = 1./(1+np.exp(-k*scores1))
    if np.mean(scores0) > 0.5:
        preds.append(0)
    else:
        preds.append(1)

In [78]:
scores0

0    0.728297
1    0.007241
2    0.212901
3    0.366604
4    0.993279
Name: 0, dtype: float64

In [60]:
scores1

0    0.475370
1    0.620596
2    0.532642
3    0.513667
4    0.377639
Name: 1, dtype: float64

In [82]:
features = np.zeros((np.unique(probssentences["doc"]).shape[0],4))
features.shape

(12500, 4)

In [80]:
from sklearn.metrics import roc_auc_score
print roc_auc_score(test1["sentiment"], preds)

0.847807030176


## Decision Tree to see rule for selecting methods

In [71]:
import numpy as np
features = np.zeros((np.unique(probssentences_train["doc"]).shape[0],4))
for doc in np.unique(probssentences_train["doc"]):
    inx = np.where(probssentences_train["doc"] == doc)[0]
    features[doc,0] = np.mean(probssentences_train[0][inx])
    features[doc,1] = np.min(probssentences_train[0][inx])
    features[doc,2] = np.max(probssentences_train[0][inx])
    features[doc,3] = np.std(probssentences_train[0][inx])

In [73]:
print features[0]

[ 0.80586817  0.1606998   0.99999881  0.29508215]


In [74]:
import pandas as pd
print len(features)
df = pd.DataFrame(features, columns=['mean0', 'min0', 'max0', 'stdev0'])
print df.shape
print df
print train1['sentiment']

12500
(12500, 4)
              mean0          min0          max0    stdev0
0      8.058682e-01  1.606998e-01  9.999988e-01  0.295082
1      8.791704e-01  5.000000e-01  9.999990e-01  0.174256
2      1.488826e-01  2.734389e-03  5.000000e-01  0.183704
3      1.111595e-01  1.353573e-03  7.157311e-01  0.229426
4      5.881894e-01  2.848751e-02  9.584527e-01  0.371201
5      8.364097e-01  3.697879e-01  9.999999e-01  0.220170
6      6.062087e-02  2.946757e-06  2.226341e-01  0.073559
7      9.652951e-01  8.575087e-01  9.999853e-01  0.046058
8      5.940239e-01  2.136570e-02  9.933403e-01  0.353413
9      1.732212e-01  2.013724e-02  5.724950e-01  0.230984
10     1.366007e-01  6.898661e-07  4.413758e-01  0.158962
11     1.604012e-03  5.583916e-08  4.625352e-03  0.001783
12     7.890698e-01  2.529315e-01  1.000000e+00  0.240222
13     8.513217e-01  3.925660e-01  9.999183e-01  0.180865
14     9.000719e-01  3.302349e-01  1.000000e+00  0.204052
15     4.147434e-01  6.910395e-06  9.095888e-01  0.4001

In [75]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
#criterion = 'entropy
clf = clf.fit(features, train1["sentiment"])

In [77]:
#get column names into an array
with open('treeGini.dot', 'w') as dotfile:
    tree.export_graphviz(
        clf,
        dotfile,
        feature_names=['mean0', 'min0', 'max0', 'stdev0'],
        max_depth=4,
        class_names=['negative', 'positive']
        )

## Attempts to visualize tree

In [23]:
tree.export_graphviz(clf, out_file = 'tree.dot')

from sklearn.externals.six import StringIO  
import pydotplus
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data) 
graph = pydotplus.graph_from_dot_data(dot_data.getvalue()) 
graph.write_pdf("tree.pdf") 

InvocationException: Program terminated with status: 1. stderr follows: Format: "pdf" not recognized. Use one of: canon cmap cmapx cmapx_np dot eps fig gv imap imap_np ismap pic plain plain-ext pov ps ps2 svg svgz tk vml vmlz xdot xdot1.2 xdot1.4


In [30]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn import tree

iris = load_iris()
test_idx = [0, 50, 100]

# training data
train_target = np.delete(iris.target, test_idx)
train_data = np.delete(iris.data, test_idx, axis=0)

# testing data
test_target = iris.target[test_idx]
test_data = iris.data[test_idx]

clf = tree.DecisionTreeClassifier()
clf.fit(train_data, train_target)

print test_target
print clf.predict(test_data) 

# viz code
from sklearn.externals.six import StringIO
import pydot
dot_data = StringIO()
tree.export_graphviz(clf,
        out_file=dot_data,
        feature_names=iris.feature_names,
        class_names=iris.target_names,
        filled=True, rounded=True,
        impurity=False)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
print type(graph)
graph.write_pdf("iris.pdf")

[0 1 2]
[0 1 2]
<class 'pydotplus.graphviz.Dot'>


InvocationException: Program terminated with status: 1. stderr follows: Format: "pdf" not recognized. Use one of: canon cmap cmapx cmapx_np dot eps fig gv imap imap_np ismap pic plain plain-ext pov ps ps2 svg svgz tk vml vmlz xdot xdot1.2 xdot1.4


In [10]:
def get_code(tree, feature_names):
        left      = tree.tree_.children_left
        right     = tree.tree_.children_right
        threshold = tree.tree_.threshold
        features  = [feature_names[i] for i in tree.tree_.feature]
        value = tree.tree_.value

        def recurse(left, right, threshold, features, node):
                if (threshold[node] != -2):
                        print "if ( " + features[node] + " <= " + str(threshold[node]) + " ) {"
                        if left[node] != -1:
                                recurse (left, right, threshold, features,left[node])
                        print "} else {"
                        if right[node] != -1:
                                recurse (left, right, threshold, features,right[node])
                        print "}"
                else:
                        print "return " + str(value[node])

        recurse(left, right, threshold, features, 0)

In [16]:
get_code(clf, cols)

if ( max <= 12220.5 ) {
if ( max <= 12076.5 ) {
if ( max <= 12069.5 ) {
if ( min <= 1.5 ) {
return [[ 2.  0.]]
} else {
if ( min <= 3.5 ) {
return [[ 0.  2.]]
} else {
if ( mean <= 9669.5 ) {
if ( min <= 9658.5 ) {
if ( max <= 7739.5 ) {
if ( max <= 7085.5 ) {
if ( min <= 7080.5 ) {
if ( max <= 3750.5 ) {
if ( max <= 3713.5 ) {
if ( min <= 3669.5 ) {
if ( max <= 3615.5 ) {
if ( min <= 3608.5 ) {
if ( max <= 3604.5 ) {
if ( min <= 3600.5 ) {
if ( max <= 3598.5 ) {
if ( mean <= 3582.5 ) {
if ( max <= 3568.5 ) {
if ( min <= 3518.5 ) {
if ( max <= 3000.5 ) {
if ( min <= 2852.5 ) {
if ( max <= 2824.5 ) {
if ( max <= 2820.5 ) {
if ( max <= 2816.5 ) {
if ( max <= 2814.5 ) {
if ( min <= 2812.5 ) {
if ( min <= 2760.5 ) {
if ( mean <= 2756.5 ) {
if ( min <= 523.5 ) {
if ( max <= 503.5 ) {
if ( mean <= 500.5 ) {
if ( max <= 385.5 ) {
if ( max <= 375.5 ) {
if ( max <= 371.5 ) {
if ( mean <= 363.5 ) {
if ( max <= 360.5 ) {
if ( max <= 143.5 ) {
if ( min <= 117.5 ) {
if ( max <= 112.5 ) {
if ( max <

In [10]:
from sklearn.externals.six import StringIO  
import pydot
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
print type(graph)
#graph.write_pdf("tree.pdf")

<type 'list'>


In [93]:
clf.feature_importances_

array([ 0.3460855 ,  0.32288789,  0.33102661,  0.        ])

In [92]:
from IPython.display import Image  
from sklearn.externals.six import StringIO 
import pydot 
dot_data = StringIO()  
tree.export_graphviz(clf, out_file=dot_data,  
                         #feature_names=iris.feature_names,  
                         #class_names=[0,1],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())  

KeyboardInterrupt: 

In [9]:
import numpy as np

estimator = clf

# The decision estimator has an attribute called tree_  which stores the entire
# tree structure and allows access to low level attributes. The binary tree
# tree_ is represented as a number of parallel arrays. The i-th element of each
# array holds information about the node `i`. Node 0 is the tree's root. NOTE:
# Some of the arrays only apply to either leaves or split nodes, resp. In this
# case the values of nodes of the other type are arbitrary!
#
# Among those arrays, we have:
#   - left_child, id of the left child of the node
#   - right_child, id of the right child of the node
#   - feature, feature used for splitting the node
#   - threshold, threshold value at the node
#

# Using those arrays, we can parse the tree structure:

n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold


# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %ss else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

# First let's retrieve the decision path of each sample. The decision_path
# method allows to retrieve the node indicator functions. A non zero element of
# indicator matrix at the position (i, j) indicates that the sample i goes
# through the node j.

node_indicator = estimator.decision_path(X_test)

# Similarly, we can also have the leaves ids reached by each sample.

leave_id = estimator.apply(X_test)

# Now, it's possible to get the tests that were used to predict a sample or
# a group of samples. First, let's make it for the sample.

sample_id = 0
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:
    if leave_id[sample_id] != node_id:
        continue

    if (X_test[sample_id, feature[node_id]] <= threshold[node_id]):
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print("decision id node %s : (X[%s, %s] (= %s) %s %s)"
          % (node_id,
             sample_id,
             feature[node_id],
             X_test[i, feature[node_id]],
             threshold_sign,
             threshold[node_id]))

# For a group of samples, we have the following common node.
sample_ids = [0, 1]
common_nodes = (node_indicator.toarray()[sample_ids].sum(axis=0) ==
                len(sample_ids))

common_node_id = np.arange(n_nodes)[common_nodes]

print("\nThe following samples %s share the node %s in the tree"
      % (sample_ids, common_node_id))
print("It is %s %% of all nodes." % (100 * len(common_node_id) / n_nodes,))

The binary tree structure has 12263 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 2] <= 12220.5s else to node 11990.
	node=1 test node: go to node 2 if X[:, 1] <= 12076.5s else to node 11817.
		node=2 test node: go to node 3 if X[:, 0] <= 12069.5s else to node 11812.
			node=3 test node: go to node 4 if X[:, 1] <= 1.5s else to node 5.
				node=4 leaf node.
				node=5 test node: go to node 6 if X[:, 1] <= 3.5s else to node 7.
					node=6 leaf node.
					node=7 test node: go to node 8 if X[:, 0] <= 9669.5s else to node 9431.
						node=8 test node: go to node 9 if X[:, 1] <= 9658.5s else to node 9422.
							node=9 test node: go to node 10 if X[:, 2] <= 7739.5s else to node 7563.
								node=10 test node: go to node 11 if X[:, 1] <= 7085.5s else to node 6954.
									node=11 test node: go to node 12 if X[:, 1] <= 7080.5s else to node 6953.
										node=12 test node: go to node 13 if X[:, 1] <= 3750.5s else to node 3714.
											node=13 test nod

/home/sarahwie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:53: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/sarahwie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:49: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


																						node=9579 test node: go to node 9580 if X[:, 1] <= 9817.5s else to node 9581.
																							node=9580 leaf node.
																							node=9581 leaf node.
																					node=9582 test node: go to node 9583 if X[:, 2] <= 9822.5s else to node 9584.
																						node=9583 leaf node.
																						node=9584 test node: go to node 9585 if X[:, 2] <= 9824.5s else to node 9586.
																							node=9585 leaf node.
																							node=9586 test node: go to node 9587 if X[:, 2] <= 9825.5s else to node 9588.
																								node=9587 leaf node.
																								node=9588 leaf node.
										node=9589 leaf node.
									node=9590 test node: go to node 9591 if X[:, 2] <= 9834.5s else to node 9592.
										node=9591 leaf node.
										node=9592 test node: go to node 9593 if X[:, 1] <= 9835.5s else to node 9594.
											node=9593 leaf node.
											node=9594 test node: go to node 9595 if X[:, 1] <= 9837

AttributeError: 'DecisionTreeClassifier' object has no attribute 'decision_path'